This tutorial shows how to run the BASC pipeline. This tutorial only shows a limited subset of available options. See the [documentation](http://niak.simexp-lab.org/pipe_basc.html) of the pipeline for a more comprehensive list of options. The script of the tutorial can be downloaded [here](https://raw.githubusercontent.com/SIMEXP/niak_tutorials/master/basc/niak_tutorial_basc.m). 

# Prepare input files

## fMRI

First download a small pre-processed fMRI dataset, with a structural scan.

In [1]:
clear
path_data = [pwd filesep];
[status,msg,data_fmri] = niak_wget('cobre_lightweight20_nii');

When starting from fMRI data preprocessed with NIAK, it is possible to use niak_grab_fmri_preprocess on the output folder to collect the file names, as described in the [pipeline documentation](http://niak.simexp-lab.org/pipe_basc.html). In this case, we explicitely a few files:

In [2]:
file_pheno = [data_fmri.path filesep 'phenotypic_data.tsv.gz'];
tab = niak_read_csv_cell(file_pheno);
list_subject = tab(2:end,1);
files_in = struct;
for ss = 1:5
    files_in.data.(list_subject{ss}).sess1.rest = [data_fmri.path filesep 'fmri_' list_subject{ss} '.nii.gz'];
end

## Analysis mask

The pipeline also needs to have one mask of brain areas. The first step of BASC is to use a region growing algorithm to reduce the spatial dimension of fMRI data to about 1000 small regions (called atoms). This stage is very memory hungry, because it runs on a concatenation of all time series from all subjects. To save memory, the region growing is run on each area independently. Typically, the pipeline uses the AAL parcellation for that purpose (or, more precisely, a tweaked version of the AAL adapted to the MNI ICBM non-linear template, see the [doc](http://niak.simexp-lab.org/pipe_region_growing.html#publication-guidelines)). The NIAK preprocessing pipeline generates this mask, which is automatically grabbed by `niak_grab_fmri_preprocess`. But here we are using low resolution data which do not come with a mask, so we are going to grab the AAL template from NIAK and resample it at the right resolution.  

In [3]:
%% Prepare the analysis mask
% load global niak variables
niak_gb_vars
% the AAL template in niak
in.source = [GB_NIAK.path_niak filesep 'template' filesep 'roi_aal_3mm.mnc.gz'];
% Use fMRI data from the first subject as target
in.target = files_in.data.(list_subject{1}).sess1.rest;
% Where to write the resampled mask 
out = [path_data 'roi_aal_cobre.nii.gz'];
% resample the data
niak_brick_resample_vol(in,out);

Copying or converting file /sandbox/home/git/niak_tutorials/basc/cobre_lightweight20/fmri_40061.nii.gz to /tmp/niak_tmp_509482616_roi_aal_cobre/target.mnc

 Reading source volume information /usr/local/niak//template/roi_aal_3mm.mnc.gz ...

 Reading target volume information /tmp/niak_tmp_509482616_roi_aal_cobre/target.mnc ...

 Resampling source on target ...
Transforming slices:..........................Done
Cleaning temporary files
Done!
Copying or converting file /tmp/niak_tmp_509482616_roi_aal_cobre/resample.mnc to /sandbox/home/git/niak_tutorials/basc/roi_aal_cobre.nii.gz
Deleting folder '/tmp/niak_tmp_509482616_roi_aal_cobre/' 
ans =

  scalar structure containing the fields:

    source = /usr/local/niak//template/roi_aal_3mm.mnc.gz
    target = /sandbox/home/git/niak_tutorials/basc/cobre_lightweight20/fmri_40061.nii.gz
    transformation = 
    transformation_stereo = gb_niak_omitted



Now we just need to tell the pipeline where to find the mask: 

In [4]:
% Specify the mask of brain areas for the pipeline
files_in.areas = out;

# Set the options of the pipeline
Now we set up where to store the results:

In [5]:
% Where to store the results
opt.folder_out = [path_data 'basc'];

This next parameter sets the size at which the region growing process stops (in mm$^3$). This does indirectly set the number of parcels covering the gray matter. A threshold of 1000 mm$^3$ will give about 1000 regions on the grey matter. Here we are going to reduce the dimension even more to speed up the pipeline.

In [8]:
% the size of the regions, when they stop growing. 
opt.region_growing.thre_size = 2000; 

The following parameter sets the values that will be explored regarding the number of clusters. A more exhaustive search, up to 500 clusters, could for example use an irregular grid like `[10:10:100 120:20:200 240:40:500]`. Also note that for each number of clusters, a number of solutions around those values, with different numbers being used at the individual and group levels, as well as the final consensus group partitions.  

In [9]:
% Search for stable clusters in the range 10 to 30
opt.grid_scales = [10:10:30]'; 

The following parameter is used to generate stability maps, and consensus partitions. Although stability is assessed over a wide range of parameters, those maps are only generated for select numbers in order to save disk space. For each set of results, three (integer) parameters actually need to be provided: the number of clusters at the individual level, at the group level and at the final consensus level. Each row will define a new set of results. We could for example use `opt.scales_maps = [10 10 10; 20 20 20];` to generate maps using the same number of clusters (5 and 10) at all three levels. For now we will leave that empty, which means we will not be generating any map. A first run of the pipeline is going to give us insights in the stable cluster solution, and provide data-driven suggestions about what number(s) of clusters to use. 

In [10]:
% Scale parameters to generate stability maps and consensus clusters
opt.scales_maps = [];

The following parameters control the number of bootstrap samples used to assess the stability of the clustering both at the individual level, and the group level. Note that with 100 samples, the confidence interval on the stability measures is $\pm 0.1$, while with 500 samples it reaches $\pm 0.05$. Here we will use only a few (20 samples) to speed-up the pipeline. 

In [11]:
% Number of bootstrap samples at the individual level.
opt.stability_tseries.nb_samps = 20;
% Number of bootstrap samples at the group level. 
opt.stability_group.nb_samps = 20; 

The final set of flags tell the pipeline which level of the pipeline to run. BASC can be used at the individual level, for each subject independently, or at the group level, to find a parcellation that is the consensus of those generated across all subjects and bootstrap replications of individual fMRI time series. In addition, BASC can generate so-called "mixed parcellation", which individual parcellations generated using the group parcellation as an initialization. It is possible to turn on/off the generation of consensus parcellations and stability map for each level of the pipeline (individual, group or mixed): 

In [12]:
% Generate maps/time series at the individual level
opt.flag_ind = false;   
% Generate maps/time series at the mixed level (group-level networks mixed with individual stability matrices).
opt.flag_mixed = false; 
% Generate maps/time series at the group level
opt.flag_group = true;  

# Running the pipeline

Now it is time to run the pipeline. As explained above, we will not at this stage generate any map or parcellation, but we will use the outputs to select the scales that we will explore in more details. The instruction to run the pipeline is: 

In [14]:
niak_pipeline_stability_rest(files_in,opt);


Logs will be stored in /sandbox/home/git/niak_tutorials/basc/basc/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.15 sec
Setting up the to-do list ...
   I found 145 job(s) to do.
I could not find any log file. This pipeline has not been started (yet?). Press CTRL-C to cancel.
Deamon started on 12-Mar-2017 05:44:46
12-Mar-2017 05:44:46 Starting the pipeline manager...
12-Mar-2017 05:44:46 Starting the garbage collector...
12-Mar-2017 05:44:46 Starting worker number 1...
12-Mar-2017 05:44:46 Starting worker number 2...
12-Mar-2017 05:44:46 Starting worker number 3...
12-Mar-2017 05:44:46 Starting worker number 4...

Pipeline started on 12-Mar-2017 05:44:47
user: , host: 2080f78652f8, system: unix
****************************************
12-Mar-2017 05:44:48 mask_areas                     submitted  (1 run | 0 fail | 0 done | 144 left)
12-Mar-2017 05:44:48 mask_areas                     finished   (0 run | 0 f

Now that the pipeline has successfully completed, we can check a .csv file generated as part of the outpus. This .csv file list the number of clusters identified as "representative" of all stable solutions across the grid of clustering parameters we selected, using a method called MSTEPS. 

In [15]:
file_msteps = [opt.folder_out filesep 'stability_group' filesep 'msteps_group_table.csv'];
[tab,lx,ly] = niak_read_csv(file_msteps);
tab

tab =

   10    7    6
   20   16   15
   30   30   38



So each line is a number of clusters at the individual/group/consensus levels. Here MSTEPS has selected three scales as appropriate to summarize all tested numbers of clusters. Note that, even though we selected only 10, 20 and 30 in our grid, that grid is strictly used only at the individual level. At the group and consensus levels, other parameters are explored, in the neighbourhood of the number of individual clusters. We can now set these parameters in the pipeline to generate stability maps and parcellations:

In [16]:
opt.scales_maps = tab;

We can finally restart the pipeline. It will pick up where it left, and only deal with the generation of the maps. 

In [17]:
niak_pipeline_stability_rest(files_in,opt);


Logs will be stored in /sandbox/home/git/niak_tutorials/basc/basc/logs/
Generating dependencies ...
   Percentage completed :  0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100- 0.23 sec
Setting up the to-do list ...
   I found 6 job(s) to do, and 145 job(s) already completed.
Deamon started on 12-Mar-2017 05:55:41
12-Mar-2017 05:55:41 Starting the pipeline manager...
12-Mar-2017 05:55:41 Starting the garbage collector...
12-Mar-2017 05:55:41 Starting worker number 1...
12-Mar-2017 05:55:41 Starting worker number 2...
12-Mar-2017 05:55:41 Starting worker number 3...
12-Mar-2017 05:55:41 Starting worker number 4...

Pipeline started on 12-Mar-2017 05:55:43
user: , host: 2080f78652f8, system: unix
****************************************
12-Mar-2017 05:55:44 stability_maps_group_sci10_scg7_scf6     submitted  (1 run | 0 fail | 145 done | 5 left)
12-Mar-2017 05:55:44 figure_stability_group_sci10_scg7_scf6   submitted  (2 run | 0 fail | 145 done | 4 left)
12-Mar-2017 05:55:44 

That's all for now! Check other tutorials to learn how to explore the results of the pipeline. 